In [ ]:
import numpy as np
import pandas as pd

Print Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Web visualization

In [ ]:
import plotly.offline as py
import plotly.express as px
import cufflinks as cf
cf.set_config_file(sharing="private", offline=True, offline_connected=False)

# The Data Table Package 

In data8 you used `numpy` and `datascience`

In [ ]:
import numpy as np
from datascience import Table

## Working with Matrices

Creating a toy matrix

In [ ]:
m = np.array(np.arange(12, dtype=float)).reshape(3,4)
m

Slicing rows and columns

In [ ]:
m[:, [1,3]]

Doing linear algebra

In [ ]:
m.T @ (m + m) 

## Working with Tables

Load data

In [ ]:
t = Table.read_table("elections.csv")
t

Access columns and rows:

In [ ]:
t.column("Party")

In [ ]:
t.row(2)

Define predicates

In [ ]:
from datascience import are
t.where("Year", are.above_or_equal_to(2000))

and do many other things...

The `Table` class in the `datascience` package implements a simple DataFrame.  However, the `Table` package is not particularly optimized and doesn't support a wide range useful functionality.  In this class, we will use Pandas, a more feature rich and widely adopted DataFrame library.  

# Pandas

It is customary to import pandas as pd

In [ ]:
import pandas as pd

## Reading Data Sources

Pandas has a number of very useful file reading tools. You can see them enumerated by typing "pd.re" and pressing tab. We'll be using read_csv today. 

In [ ]:
elections = pd.read_csv("elections.csv")
elections # if we end a cell with an expression or variable name, the result will print

Read a table from a website

In [ ]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/Greenhouse_gas")
dfs[4]  # read the 5th table on the page

Read a Microsoft Excel file

In [ ]:
pd.read_excel("fossil_fuel.xlsx", sheet_name="Data2")

## Looking at only a few rows

We can use the head command to return only a few rows of a dataframe.

In [ ]:
elections.head()

In [ ]:
elections.head(3)

There is also a tail command.

In [ ]:
elections.tail(7)

A random sample of 7 entries.

In [ ]:
# Note I am seeding the sample so later stages in my
# notebook work. My favorite seed is 42
elec_sample = elections.sample(7, random_state=42) 
elec_sample

Sampling with replacement

In [ ]:
elections.sample(10, replace=True)

Sampling columns?

In [ ]:
elections.sample(2, axis=1).head()

## Utility Operations

In addition to `head`, `tail`, and `sample` the are a range of useful operations.

Shape returns the number of rows and columns.

In [ ]:
elections.shape

Size describes the number of "cells" in the dataframe

In [ ]:
elections.size

We can sort the rows by their values:

In [ ]:
elections.sort_values(["Year", "Result"])

We can rename columns:

In [ ]:
elections.rename(columns={"%": "Percent"}).head()

**Note** that the `rename` method returned a new dataframe and didn't modify the original one.  

**Most operations in Pandas are not mutating.** This produces cleaner code.  If you change something it should be stored in a new appropriately named variable.  

In [ ]:
elections.head()

Casting types.

In [ ]:
elections.astype({"Year": float}).head()

We can get summary statistics for each column

In [ ]:
elections.describe(include="all")

You can even transpose a dataframe

In [ ]:
elections.transpose()

The fact that you can take the transpose implies some interesting symmetry properties that will lead us to treat rows and columns symmetrically.  This has interesting implications on how we refer to rows and columns!

## Anatomy of a DataFrame

The DataFrame has named columns

In [ ]:
elections.head()

In [ ]:
elections.columns

The columns have types

In [ ]:
elections.dtypes

You can access a the rows as an array of lists

In [ ]:
elections.values

## Indexes (Part 1)

All dataframes have an **index**.

In [ ]:
elections

In [ ]:
elections.index

By default a RangeIndex is attached enumerating the rows.  This is shown in bold as the far left column.  Recall that we sampled the elections table.  Let's examine that sample:

In [ ]:
elec_sample

In [ ]:
elec_sample.index

Notice that the index is different.  It maintained the index of the rows in the original table.  This is really useful if we wanted to go back and relate derived tables with their original values.

You can change the index.

In [ ]:
elec_sample_iyear = elec_sample.set_index("Year")
elec_sample_iyear

In [ ]:
elec_sample_iyear.index

**Note** that the `set_index` operation is not mutating.   

In [ ]:
elections.index

In [ ]:
elections.head()

The index allows you to reference *rows* by *name*.  You will see this in a moment when we talk about slicing.  

**Note:** The index does not need to be unique.

#### The Columns are also an index

Recall that we could get the list of column names

In [ ]:
elections.columns

Notice that the return type is an index.  Recall we could transpose a DataFrame.  This is effectively swapping the row and column index:

In [ ]:
elections.transpose().index

## Accessing Rows and Columns (Slicing)

There are many ways to access rows and columns of a Pandas DataFrame.  We will spend some time reviewing most of the options.

### Accessing Columns using `[  ]`

The DataFrame class has an indexing operator [] that lets you do a variety of different things.

Just like the Table in data8 you can access columns using the square `[  ]` brakets.

In [ ]:
elec_sample

You can pass a list of columns names:

In [ ]:
elec_sample[ ["Candidate", "Year"] ] # space added to show the list

In [ ]:
elec_sample[["Candidate", "Year", "Result"]] # No space is more standard

If you pass a list with a single element you get back a DataFrame.

In [ ]:
elec_sample[["Candidate"]]

## Series

If you pass a single item instead of a list you get back a `Series`

In [ ]:
party = elec_sample["Party"]
party

When accessing a single column we get back a `pd.Series` object

In [ ]:
type(party)

The series object represents a single column (or row) of data.  The `Series` object has a `index`, a `name`, and `values`.  A series can be thought of as a map.

In [ ]:
party.index

In [ ]:
party.name

In [ ]:
party.values

We can convert a Series into a DataFrame

In [ ]:
party.to_frame()

Series act like numpy arrays and support most numpy operations

In [ ]:
year = elec_sample["Year"]
year

In [ ]:
year.mean()

Apply numpy opeations:

In [ ]:
np.sin(year * 3)

We can sort the `Series` by value or by index.

In [ ]:
np.sin(year * 3).sort_values()

In [ ]:
np.sin(year * 3).sort_index()

### Counting unique values

Series also has a very useful function `.value_counts()` which allows us to compute the number of occurences of each unique value.

In [ ]:
year.value_counts()

In [ ]:
party_counts = elections['Party'].value_counts()
party_counts

Note that in each case we also got back a series and these series (like all series) are maps from index to value.

In [ ]:
party_counts.index

In [ ]:
party_counts.values

In [ ]:
party_counts["Independent"]

### Indexes allow us to relate data

Notice how in call cases I keep track of the **index** making it possible to relate this data back to the sample and even the original table. 

For example, in the following we create a new series with the name weird and join it back with the original data. 

In [ ]:
weird = np.sin(year * 3).rename("weird")
weird

Here we use the Pandas join operation. This joins on the index.  You will learn  more about this next and the more general `merge` operation next week.

In [ ]:
elections.join(weird) 

In [ ]:
weird.to_frame().join(elections)

What kind of join is this?

## Column Assignment (Mutating)

You can modify and even add columns using the square brackets `[ ]` 

In [ ]:
tmp = elec_sample.copy()
tmp["Year"] = tmp["Year"] * -1 + .5
tmp

Adding a new column by assignment:

In [ ]:
tmp["Corrected Year"] = tmp["Year"] * -1 + .5
tmp

In [ ]:
tmp["Random Numbers"] = np.random.randn(tmp.shape[0])
tmp

### Accessing by rows and columns by index `.loc`

You can access rows and columns of a DataFrame by **name** using the `.loc[ ]` syntax.

In [ ]:
elec_sample

In [ ]:
elec_sample.loc[:, ["Party", "Year"] ]

The syntax for `.loc` is:

```
  df.loc[ rows_list, column_list ]
```


We can pass a list of row names (index values):

In [ ]:
elec_sample

In [ ]:
elec_sample.loc[[1,15,9], :]

In [ ]:
elec_sample.loc[[1,15,9]]

In [ ]:
elec_sample_iyear

How many rows will this call return

In [ ]:
elec_sample_iyear.loc[[2004, 1980], :]

Loc also supports slicing (for all types, including numeric and string labels!). Note that the slicing for loc is **inclusive**, even for numeric slices.  In general, avoiding range slicing with `.loc`.  

In [ ]:
elections.loc[0:4, 'Candidate':'Year']

Keep in mind that the ranges need to be over the index values and not the locations.

Note the index values need to have well defined contiguous ranges.

In [ ]:
elec_sample_iyear.sort_index().loc[1980:2004, 'Candidate':'Party']

If we provide only a single label for the column argument, we get back a Series just as with regular `[ ]` indexing.

In [ ]:
elec_sample.loc[:, 'Candidate']

If we want a data frame instead and don't want to use to_frame, we can provde a list containing the column name.

In [ ]:
elec_sample.loc[:, ['Candidate']]

We can also select a single row.  Notice that in this case we also get back a `Series` where the index is the set of columns.

In [ ]:
obama_row = elec_sample_iyear.loc[2008, :]
obama_row

In [ ]:
obama_row.name

In [ ]:
obama_row.index

In [ ]:
obama_row.values

It is worth noting that the `Series` also functions like a map from the index to the values.

In [ ]:
obama_row["Party"]

If we omit the column argument altogether, the default behavior is to retrieve all columns. 

In [ ]:
elections.set_index("Year").loc[[2008, 2012]]

## Indexing a Single Value 

What happens if you give scalar arguments for the requested rows AND columns. The answer is that you get back just a single value.

In [ ]:
elections.loc[0, 'Candidate']

## Boolean Array Selection (WHERE in SQL)

The `.loc[]` and also `[ ]` support arrays of booleans as an input. In this case, the array must be exactly as long as the number of rows. The result is a filtered version of the data frame, where only rows corresponding to True appear.

In [ ]:
elec_sample

In [ ]:
elec_sample.loc[[False, False, False, False, True, False, False]]

You can also pass the same arguments to the `[ ]` operator.

In [ ]:
elec_sample[[False, False, False, False, True, False, False]]

One very common task in Data Science is filtering. Boolean Array Selection is one way to achieve this in Pandas. We start by observing logical operators like the equality operator can be applied to Pandas Series data to generate a Boolean Array. For example, we can compare the 'Result' column to the String 'win':

In [ ]:
elections.head(5)

In [ ]:
iswin = elections['Result'] == 'win'
iswin

The output of the logical operator applied to the Series is another Series with the same name and index, but of datatype boolean. The entry at row #i represents the result of the application of that operator to the entry of the original Series at row #i.

Such a boolean Series can be used as an argument to the [] operator. For example, the following code creates a DataFrame of all election winners since 1980.

In [ ]:
elections[iswin]

Above, we've assigned the result of the logical operator to a new variable called `iswin`. This is uncommon. Usually, the series is created and used on the same line. Such code is a little tricky to read at first, but you'll get used to it quickly.

In [ ]:
elections[elections['Result'] == 'win']

We can select multiple criteria by creating multiple boolean Series and combining them using the `&` operator.

In [ ]:
elections[
    (elections['Result'] == 'win') & 
    (elections['%'] < 50)
]

Using the logical negation `~` operator (not).  

In [ ]:
elections[
    (elections['Result'] == 'win') & 
    ~(elections['%'] < 50)
]

The | operator is the symbol for or.

In [ ]:
elections[
    ~((elections['Party'] == "Democratic") | 
        (elections['Party'] == "Republican"))
]

If we have multiple conditions (say Republican or Democratic), we can use the isin operator to simplify our code.

In [ ]:
elections[elections['Party'].isin(["Republican", "Democratic"])]

An alternate simpler way to get back a specific set of rows is to use the `query` command. 

In [ ]:
elections.query("Result == 'win' and Year < 2000")

Note, the query command needs a bit of care and cannot be applied to that contain periods or special characters. 

In [ ]:
tmp2 = elections.rename(columns={"Year": "Elec Year", "%": "%*100"})
tmp2.head()

In [ ]:
tmp2.query("`Elec Year` > 2000")

In general, I don't use the query function because of these issues. 

### Accessing by Location `iloc` (Integer-Location)

This is similar to a spreadsheet

loc's cousin iloc is very similar, but is used to access based on numerical position instead of label. For example, to access to the top 3 rows and top 3 columns of a table, we can use [0:3, 0:3]. iloc slicing is **exclusive**, just like standard Python slicing of numerical values.

In [ ]:
elec_small = elections.head(5)
elec_small

We use the the `dataframe.iloc[row_slice, column_slice]` to access specific rows and columns by their location.  For example, to access to the top 3 rows and top 3 columns of a table, we can use [0:3, 0:3]. iloc slicing is **exclusive**, just like standard Python slicing of numerical values.

In [ ]:
elec_small.iloc[0:3, 0:3]

Return the last three rows using slicing?

<details><summary>Solution</summary>
    <code>elec_small.iloc[-3:, :]</code>
</details>

In [ ]:
# code here

<br/><br/><br/>
### Caution

We will use both loc and iloc in the course. Loc is generally preferred for a number of reasons, for example: 

1. It is harder to make mistakes since you have to literally write out what you want to get.
2. Code is easier to read, because the reader doesn't have to know e.g. what column #31 represents.
3. It is robust against permutations of the data, e.g. the social security administration switches the order of two columns.

However, iloc is sometimes more convenient. We'll provide examples of when iloc is the superior choice.

<br/><br/>

## Running Python on Rows

In general, we avoid directly running python code on each row or iterating over dataframes.  It is often orders of magnitude faster to use builtin operations to modify entire columns at once.  

However, occasionally you need to apply code to the dataframe directly.  

The `apply` function executes the input function on each row (if axis=1) or each column (if axis=0):

In [ ]:
elections.apply(lambda row: row['Party'][0:3], axis=1)

You can also directly iterate over the rows:

In [ ]:
for (year, row) in elections.set_index("Year").iterrows():
    if row['Result'] == "win":
        print(f"In {year} the winner was {row['Party']}")

## Quick Challenge

Which of the following expressions return DataFrame of the first 3 Candidate and Party names for candidates that won with more than 50% of the vote.

In [ ]:
elections.iloc[[0, 3, 5], [0, 3]]

In [ ]:
elections.loc[[0, 3, 5], "Candidate":"Year"]

In [ ]:
elections.loc[elections["%"] > 50, ["Candidate", "Year"]].head(3)

In [ ]:
elections.loc[elections["%"] > 50, ["Candidate", "Year"]].iloc[0:2, :]

## Handy Properties and Utility Functions for Series and DataFrames

#### Python Operations on Numerical DataFrames and Series

Consider a series of only the vote percentages of election winners.

In [ ]:
winners = elections.query("Result == 'win'")["%"]
winners

We can perform various Python operations (including numpy operations) to DataFrames and Series.

In [ ]:
max(winners)

In [ ]:
np.mean(winners)

We can also do more complicated operations like computing the mean squared error, i.e. the average L2 loss.

In [ ]:
c = 50.38
mse = np.mean((c - winners)**2)
mse

In [ ]:
c2 = 50.35
mse2 = np.mean((c2 - winners)**2)
mse2

We can also apply mathematical operations to a DataFrame so long as it has only numerical data.

In [ ]:
(elections[["%", "Year"]] + 3).head(5)

## Baby Names Data

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

Goal 1: Find the most popular female baby names in California in 2018

<details>
    <summary>Solution</summary>
<code>
babynames[
    (babynames["Year"] == 2018) & 
    (babynames["Sex"] == "F")
].sort_values(by = "Count", ascending = False).head(20)
</code>
</details>

In [ ]:
# Solution here

Goal 2: Make a plot of how many baby girls were named Nora over the years.

<details>
    <summary>Solution</summary>
<code>
babynames[(babynames.Name == "Nora") & (babynames.Sex == "F") ].iplot(x="Year", y="Count")
</code>
</details>


In [ ]:
# Solution here